In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [4]:
(features_train, target_train), (features_test, target_test) = mnist.load_data()
num_labels = len(np.unique(target_train))

11493376/11490434 [==============================] - 0s 0us/step


In [5]:
target_train = to_categorical(target_train)
target_test = to_categorical(target_test)

In [6]:
image_size = features_train.shape[1]

In [7]:
features_train= np.reshape(features_train,[-1, image_size, image_size, 1])
features_test = np.reshape(features_test,[-1, image_size, image_size, 1])
features_train = features_train.astype('float32') / 255
features_test = features_test.astype('float32') / 255

In [8]:
input_shape = (image_size, image_size, 1)
batch_size = 128
kernel_size = 3
filters = 64
dropout = 0.3

In [9]:
# use functional API to build cnn layers
inputs = Input(shape=input_shape)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(inputs)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
y = MaxPooling2D()(y)
y = Conv2D(filters=filters,
           kernel_size=kernel_size,
           activation='relu')(y)
# image to vector before connecting to dense layer
y = Flatten()(y)
# dropout regularization
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

# build the model by supplying inputs/outputs
model = Model(inputs=inputs, outputs=outputs)
# network model in text
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0     

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [11]:
model.fit(features_train,
          target_train,
          validation_data=(features_test, target_test),
          epochs=20,
          batch_size=batch_size)


Epoch 1/20
469/469 [==============================] - 41s 86ms/step - loss: 0.6186 - accuracy: 0.8031 - val_loss: 0.0680 - val_accuracy: 0.9783
Epoch 2/20
469/469 [==============================] - 40s 84ms/step - loss: 0.0801 - accuracy: 0.9747 - val_loss: 0.0381 - val_accuracy: 0.9878
Epoch 3/20
469/469 [==============================] - 39s 83ms/step - loss: 0.0566 - accuracy: 0.9827 - val_loss: 0.0288 - val_accuracy: 0.9916
Epoch 4/20
469/469 [==============================] - 39s 83ms/step - loss: 0.0465 - accuracy: 0.9846 - val_loss: 0.0261 - val_accuracy: 0.9909
Epoch 5/20
469/469 [==============================] - 39s 83ms/step - loss: 0.0394 - accuracy: 0.9880 - val_loss: 0.0262 - val_accuracy: 0.9914
Epoch 6/20
469/469 [==============================] - 39s 84ms/step - loss: 0.0328 - accuracy: 0.9890 - val_loss: 0.0233 - val_accuracy: 0.9918
Epoch 7/20
469/469 [==============================] - 39s 83ms/step - loss: 0.0285 - accuracy: 0.9908 - val_loss: 0.0239 - val_accuracy:

In [12]:
score = model.evaluate(features_test,
                       target_test,
                       batch_size=batch_size,
                       verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))


Test accuracy: 99.3%
